In [3]:
# Import data from CSV file
# Practice-level data, range May 2016 to April 2018
# Other category (as per BNF 040304) contains venlafaxine, mirtazapine, and duloxetine

import pandas as pd
import numpy as np
import os
os.chdir("/Users/paulbogowicz/Documents/Research/Oxford University/OpenPrescribing - antidepressants/Data")

df1 = pd.read_csv("antidepressant_12mo_practice_level_summary_08.08.18.csv")
df1 = df1.rename(columns={"practice":"Practice","month":"Month","total_items":"Total","total_cost":"Cost","MAOi":"MAOI"})
print("No of practices: ", df1["Practice"].nunique())
df1.head()

No of practices:  7143


,Practice,Other,Venlafaxine,Mirtazapine,Duloxetine,MAOI,Citalopram,Fluoxetine,Fluvoxamine,Sertraline,...,Tricyclics,Trimipramine,Dosulepin,Amitriptyline,Trazodone,Lofepramine,Maprotiline,Mianserin,Total,Cost
0,A81001,1840,456,1229,143,0,1041,471,0,1535,...,1470,7,67,1007,139,79,0,0,6546,19161.69614
1,A81002,7248,2062,3631,1492,0,4929,4761,0,7823,...,7375,14,342,5716,914,103,0,0,33146,78605.05954
2,A81004,2956,619,1480,831,7,1476,1865,4,3379,...,3252,7,65,2713,295,26,0,0,13375,50695.01171
3,A81005,2359,648,1391,307,13,2258,987,0,3078,...,1959,0,0,1626,179,69,0,0,11174,23972.33662
4,A81006,4941,1169,2904,863,1,2757,3829,11,7995,...,6428,2,150,5586,392,49,0,0,26400,63894.24967


### Practice-level calculations

In [4]:
# Calculate class totals

df1["Others"] = df1["Other"] - df1["Venlafaxine"] - df1["Duloxetine"] + df1["Maprotiline"] + df1["Mianserin"] + df1["Trazodone"]
df1["SSRI"] = df1.loc[:,["Citalopram","Escitalopram","Fluoxetine","Fluvoxamine","Paroxetine","Sertraline"]].sum(axis = 1)
df1["SNRI"] = df1.loc[:,["Duloxetine","Venlafaxine"]].sum(axis = 1)
df1["Tricyclic"] = df1["Tricyclics"] - df1["Maprotiline"] - df1["Mianserin"] - df1["Trazodone"]
df1["Test"] = df1["Total"] - df1["Others"] - df1["SSRI"] - df1["SNRI"] - df1["Tricyclic"] - df1["MAOI"]
print(df1.loc[:,"Test"].sum(axis=0))

0


In [7]:
# Calculate percentages for each class

df1["SSRI %"] = 100*df1["SSRI"]/df1["Total"]
df1["SNRI %"] = 100*df1["SNRI"]/df1["Total"]
df1["MAOI %"] = 100*df1["MAOI"]/df1["Total"]
df1["Tricyclic %"] = 100*df1["Tricyclic"]/df1["Total"]
df1["Other %"] = 100*df1["Others"]/df1["Total"]

# Calculate percentages for top 10 most prescribed (2017) and others of interest

df1["Amitriptyline %"] = 100*df1["Amitriptyline"]/df1["Total"]
df1["Citalopram %"] = 100*df1["Citalopram"]/df1["Total"]
df1["Duloxetine %"] = 100*df1["Duloxetine"]/df1["Total"]
df1["Escitalopram %"] = 100*df1["Escitalopram"]/df1["Total"]
df1["Fluoxetine %"] = 100*df1["Fluoxetine"]/df1["Total"]
df1["Mirtazapine %"] = 100*df1["Mirtazapine"]/df1["Total"]
df1["Paroxetine %"] = 100*df1["Paroxetine"]/df1["Total"]
df1["Sertraline %"] = 100*df1["Sertraline"]/df1["Total"]
df1["Trazodone %"] = 100*df1["Trazodone"]/df1["Total"]
df1["Venlafaxine %"] = 100*df1["Venlafaxine"]/df1["Total"]
df1["Dosulepin %"] = 100*df1["Dosulepin"]/df1["Total"]
df1["Trimipramine %"] = 100*df1["Trimipramine"]/df1["Total"]

# Troubleshooting code - to identify those prescribing 100% SSRIs
# for i in range(0,7143):
#     if df1.loc[i,"SSRI %"] == 100.0:
#         print(df1.loc[i])

In [18]:
# Calculate summary stats for different classes of antidepressants, at practice level

import scipy.stats as stats

# Calculate summary statistics, select columns, and transpose

smry = df1.describe() # summary statistics
smry = smry.loc[:,["SSRI %", "SNRI %", "Tricyclic %", "MAOI %","Other %"]]
smry = smry.transpose()

# Calculate IQR and kurtosis
smry["IQR"] = smry.loc[:,"75%"] - smry.loc[:,"25%"]
k = pd.Series(name="Kurtosis",index=["SSRI %", "SNRI %", "Tricyclic %", "MAOI %","Other %"])
for i in range(0,len(k)):
    k[i] = stats.kurtosis(df1[k.index[i]])
kdf = k.to_frame()
smry["Kurtosis"] = kdf["Kurtosis"]

# Select columns and print
smry2 = pd.DataFrame(smry,columns=["mean", "std", "50%","25%","75%", "IQR", "Kurtosis"])
smry2 = smry2.rename(columns={'mean': 'Mean', '25%': 'Lower Quartile', '50%': 'Median', '75%': 'Upper Quartile',"std":"Std Dev"})
smry2


,Mean,Std Dev,Median,Lower Quartile,Upper Quartile,IQR,Kurtosis
SSRI %,54.304211,6.750974,54.389966,50.360798,58.386517,8.025718,3.602023
SNRI %,8.924691,3.140442,8.668602,6.825307,10.733488,3.908181,1.459000
Tricyclic %,22.619476,5.704843,22.379569,19.092694,25.617900,6.525206,9.127806
MAOI %,0.063606,0.163279,0.000000,0.000000,0.067507,0.067507,132.448706
Other %,14.088016,5.264474,13.625922,10.540779,16.917887,6.377108,7.703261


In [15]:
# Calculate summary stats for top 10 most prescribed antidepressants (2017), at practice level

# Calculate summary statistics, select columns, and transpose

smry3 = df1.describe() # summary statistics
smry3 = smry3.loc[:,["Citalopram %", "Amitriptyline %", "Sertraline %", "Mirtazapine %","Fluoxetine %","Venlafaxine %","Duloxetine %","Paroxetine %","Trazodone %","Escitalopram %"]]
smry3 = smry3.transpose()

# Calculate IQR and kurtosis
smry3["IQR"] = smry3.loc[:,"75%"] - smry3.loc[:,"25%"]
k2 = pd.Series(name="Kurtosis",index=["Citalopram %", "Amitriptyline %", "Sertraline %", "Mirtazapine %","Fluoxetine %","Venlafaxine %","Duloxetine %","Paroxetine %","Trazodone %","Escitalopram %"])
for i in range(0,len(k2)):
    k2[i] = stats.kurtosis(df1[k2.index[i]])
kdf2 = k2.to_frame()
smry3["Kurtosis"] = kdf2["Kurtosis"]

# Select columns and print
smry4 = pd.DataFrame(smry3,columns=["mean", "std", "50%","25%","75%", "IQR", "Kurtosis"])
smry4 = smry4.rename(columns={'mean': 'Mean', '25%': 'Lower Quartile', '50%': 'Median', '75%': 'Upper Quartile','std':"Std Dev"})
smry4

,Mean,Std Dev,Median,Lower Quartile,Upper Quartile,IQR,Kurtosis
Citalopram %,21.574343,6.441841,21.315193,17.282590,25.494070,8.211481,6.516955
Amitriptyline %,19.515538,5.476911,19.196272,16.028174,22.366300,6.338126,8.560337
Sertraline %,19.423017,5.634343,19.207003,15.651114,22.838097,7.186983,1.260870
Mirtazapine %,12.305541,4.771011,11.868162,9.069395,14.766482,5.697087,7.521337
Fluoxetine %,9.524715,3.638861,9.076479,7.172921,11.452540,4.279618,21.903466
Venlafaxine %,5.951901,2.496923,5.683730,4.222515,7.413288,3.190772,1.530776
Duloxetine %,2.972790,1.869250,2.592593,1.692049,3.858162,2.166112,4.856518
Paroxetine %,2.107575,1.332047,1.849692,1.272561,2.633470,1.360909,13.914765
Trazodone %,1.447120,1.617082,0.954965,0.379443,1.925309,1.545866,17.061365
Escitalopram %,1.639563,2.102077,1.027582,0.491607,1.996325,1.504719,32.279783


In [16]:
# Calculate summary stats for drugs that shouldn't be prescribed, at practice level

# Calculate summary statistics, select columns, and transpose

smry5 = df1.describe() # summary statistics
smry5 = smry5.loc[:,["Paroxetine %","Dosulepin %","Trimipramine %"]]
smry5 = smry5.transpose()

# Calculate IQR and kurtosis
smry5["IQR"] = smry5.loc[:,"75%"] - smry5.loc[:,"25%"]
k3 = pd.Series(name="Kurtosis",index=["Paroxetine %","Dosulepin %","Trimipramine %"])
for i in range(0,len(k3)):
    k3[i] = stats.kurtosis(df1[k3.index[i]])
kdf3 = k3.to_frame()
smry5["Kurtosis"] = kdf3["Kurtosis"]

# Select columns and print
smry6 = pd.DataFrame(smry5,columns=["mean", "std", "50%","25%","75%", "IQR", "Kurtosis"])
smry6 = smry6.rename(columns={'mean': 'Mean', '25%': 'Lower Quartile', '50%': 'Median', '75%': 'Upper Quartile','std':"Std Dev"})
smry6

,Mean,Std Dev,Median,Lower Quartile,Upper Quartile,IQR,Kurtosis
Paroxetine %,2.107575,1.332047,1.849692,1.272561,2.633470,1.360909,13.914765
Dosulepin %,1.149615,1.051267,0.919104,0.458451,1.543044,1.084593,32.777573
Trimipramine %,0.077161,0.222797,0.000000,0.000000,0.069643,0.069643,102.241729


### Code not modified beyond this point

In [4]:
#############################################
#############################################
#### Code not modified beyond this point ####
#############################################
#############################################

#prevalence data for CCGs
prevccg = pd.read_csv('diabetes_QOF_figures_by_CCG_16_17.csv',usecols={'CCG code','2016-17 Register','2016-17 Prevalence (per cent)'})
prevccg = prevccg.rename(columns={'CCG code':'CCG','2016-17 Register':'Total','2016-17 Prevalence (per cent)':'percentage'})

# population sizes by practice (use 15+ population as closest approximation; QOF figures are for 17+)
q2 = '''SELECT stat.practice, 
stat.pct_id AS CCG,
total_list_size - male_0_4 - female_0_4 - male_5_14 - female_5_14 AS list_size_15plus
FROM ebmdatalab.hscic.practice_statistics stat  
WHERE DATE(stat.month) ='2017-08-01'
'''
pop = pd.io.gbq.read_gbq(q2, GBQ_PROJECT_ID, dialect='standard',verbose=False)

#join practice populations to data, this also maps each practice to their CURRENT CCG. 
ccg = df1.merge(pop, on="practice", how="left")
#sum data to ccg level
ccg = ccg.groupby('CCG').sum().reset_index()
# join prevalence data at ccg level
ccg = ccg.merge(prevccg[["CCG","percentage","Total"]],on="CCG",how='left')
ccg = ccg.loc[~ccg['CCG'].str[-1].str.isnumeric()] ## filter out any odd CCG codes where last character is numeric

# use percentage prevalence to calculate number of diabetics, applied to practices included in data.
ccg["calc_diabetics"] = ccg["list_size_15plus"]*ccg["percentage"]/100 

# ccg level calculations
ccg["Items per diabetic"] = ccg.all_antidiab/ccg.calc_diabetics
ccg["Cost per diabetic"] = ccg.all_antidiab_cost/ccg.calc_diabetics

ccg["Metformin (%)"] = 100*ccg.Metformin/ccg.all_antidiab
ccg["Sulfonylurea (%)"] = 100*ccg.Sulfonylurea/ccg.all_antidiab
ccg["DPP-4 (%)"] = 100*ccg.DPP4/ccg.all_antidiab
ccg["TZD (%)"] = 100*ccg.TZD/ccg.all_antidiab
ccg["SGLT-2 (%)"] = 100*ccg.SGLT2/ccg.all_antidiab
ccg["GLP-1 (%)"] = 100*ccg.GLP1/ccg.all_antidiab
ccg["Non-Met Non-SU"] = ccg.all_antidiab-ccg.Metformin-ccg.Sulfonylurea
ccg["Non-Met Non-SU (%)"] = 100*ccg["Non-Met Non-SU"]/ccg.all_antidiab

# code to check differences between calculated prevalence and QOF figures
#ccg["diff"] = (ccg["Total"]-ccg["calc_diabetics"])/ccg["Total"]
#ccg.sort_values(by="diff")

print("Number of CCGs: ", ccg['CCG'].nunique())

Number of CCGs:  207


In [5]:
# print total population covered and no of diabetics
ccg[["list_size_15plus","Total"]].sum()

list_size_15plus    47897850.0
Total                3116399.0
dtype: float64

### CCG-level summary table

In [6]:
ccgsmry = ccg.describe() ## summary statistics

# kurtosis
k = pd.Series(0.1,name="Kurtosis",index=["Metformin (%)", 'Sulfonylurea (%)', 'DPP-4 (%)', 'TZD (%)','SGLT-2 (%)','GLP-1 (%)',"Non_Met_Non_SU (%)"])
for i in range(0,len(k)):
    k[i] = stats.kurtosis(df1[k.index[i]])

## calculate ranges:
iqrc = ccgsmry.loc['75%'] - ccgsmry.loc['25%']
iqrc = iqrc.rename('IQR')

##standard deviation
stdevc = ccg.std()
stdevc = stdevc.rename('Std Dev')

## append the calculated fields onto the summary table
ccgsmry= ccgsmry.append([iqrc,k,stdevc])

## select only the % columns:
ccgsmry2 = pd.DataFrame(ccgsmry,columns=['Metformin (%)','Sulfonylurea (%)','DPP-4 (%)','TZD (%)','SGLT-2 (%)','GLP-1 (%)',"Non-Met Non-SU (%)","Items per diabetic","Cost per diabetic"])

## transpose and select only stats of interest:
ccgsmry2 = ccgsmry2.transpose()
ccgsmry2 = pd.DataFrame(ccgsmry2,columns=["mean","Std Dev", "50%","25%","75%", "IQR", "Kurtosis"])

ccgsmry2 = ccgsmry2.rename(columns={'mean': 'Mean','25%': 'Lower Quartile', '50%': 'Median', '75%': 'Upper Quartile'})
ccgsmry2

,Mean,Std Dev,Median,Lower Quartile,Upper Quartile,IQR,Kurtosis
Metformin (%),55.556122,2.851447,55.187991,53.573845,57.579108,4.005263,8.035139
Sulfonylurea (%),21.602577,3.470960,21.638336,19.164894,24.040554,4.875660,5.075004
DPP-4 (%),13.502197,3.265248,13.952086,11.299162,15.993987,4.694825,2.748892
TZD (%),2.471274,1.394481,2.235040,1.517963,3.036035,1.518072,8.549002
SGLT-2 (%),4.104381,1.602967,4.214014,2.926287,5.226648,2.300361,10.274083
GLP-1 (%),2.437945,0.932301,2.376978,1.794380,2.930301,1.135921,46.578401
Non-Met Non-SU (%),22.841301,4.702541,23.625220,20.023555,26.169077,6.145522,NaN
Items per diabetic,11.604143,1.979939,11.399356,10.371183,12.844347,2.473164,NaN
Cost per diabetic,130.284937,25.282891,130.631251,113.692387,148.271753,34.579366,NaN


## Calculated savings

In [7]:
# lowest decile cost
mincost = ccg["Cost per diabetic"].quantile(0.1)
print ("Lowest decile cost: £",format(mincost, '0,.2f')," per person with diabetes") 

# calculate potential savings for each CCG above lowest decile
ccg2 = ccg
ccg2["possible min cost"] = ccg2["calc_diabetics"]*mincost
ccg2["possible saving"] = ccg2["all_antidiab_cost"] - ccg2["possible min cost"]
ccg2 = ccg2.loc[ccg2["possible saving"]>0]
print ("Total possible saving: £",format(ccg2["possible saving"].sum(),'0,.0f'))
print ("Total spend: £",format(ccg["all_antidiab_cost"].sum(),'0,.0f'))
print ("Percentage possible saving: ",format(100*ccg2["possible saving"].sum()/ccg["all_antidiab_cost"].sum(),'0,.1f'),"%")

Lowest decile cost: £ 94.87  per person with diabetes
Total possible saving: £ 112,995,874
Total spend: £ 411,836,153
Percentage possible saving:  27.4 %


In [8]:
ccg.to_csv('ccg_level_diabetes_detail_to_Aug2017.csv')